<a href="https://colab.research.google.com/github/scrapers4/Modificar_imagenes/blob/main/modificar_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab utilizado para realizar el tratamiento de las imagenes despues de descargadas.




## instalacion de cloudinary

In [ ]:
!pip install cloudinary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cloudinary: filename=cloudinary-1.33.0-py3-none-any.whl size=137406 sha256=414e0672f17fc0aadc8164cfbf8e3f1c894d6e4d96f82a3ecd5a828a7e969d82
  Stored in directory: /root/.cache/pip/wheels/1a/21/3e/af30b23ed13022e13ce960362af3dccf7bc4e0d3824b370b6e
Successfully built cloudinary


## importacion de las librerias

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import urllib
import urllib.request
import uuid
import re
import asyncio
import base64
from base64 import b64decode
import requests
from PIL import Image
from io import BytesIO
import os
from bs4 import BeautifulSoup
import numpy as np


## Funcion resize_image
### Esta funcion toma por argumento la variable imagen_original, para modificarla, Se convierte a escala de grises, se pasan los datos a una matrix numpy donde se filtra por el color blanco que tiene un valor de 255, en caso de no encontrar el valor de 255 se trabaja mediante el metodos estadisticos para obtener el valor que es mas frecuente y cercano a 255, al obtener este valor del blanco pasamos a obtenr las coordenadas del cuadro blanco para cortar la imagenen y luego multiplicar la altura y anchura por 2  y se retorna la imagen.

In [ ]:

import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


def resize_image(imagen_original):
  try:
    # Convertir la imagen a escala de grises
    imagen_gris = imagen_original.convert('L')
    # Convertir la imagen a un array numpy
    array_imagen = np.array(imagen_gris)
    # Obtener las coordenadas del cuadrado blanco
    indices_blancos = np.where(array_imagen >= 255)
    if np.size(indices_blancos) == 0:
      
      histograma, bordes = np.histogram(array_imagen, bins=10)
      valor_cercano_255 = None
      distancia_minima = float('inf')
      frecuencia_maxima = 0

      for i in range(len(bordes) - 1):
          valor_actual = bordes[i]
          distancia_actual = abs(valor_actual - 255)
          frecuencia_actual = histograma[i]
          
          if distancia_actual < distancia_minima:
              valor_cercano_255 = valor_actual
              distancia_minima = distancia_actual
              frecuencia_maxima = frecuencia_actual
          elif distancia_actual == distancia_minima and frecuencia_actual > frecuencia_maxima:
              valor_cercano_255 = valor_actual
              frecuencia_maxima = frecuencia_actual

      indices_blancos = np.where(array_imagen >= valor_cercano_255)

    # Obtener las coordenadas del cuadrado blanco

    min_x = np.min(indices_blancos[1])
    max_x = np.max(indices_blancos[1])
    min_y = np.min(indices_blancos[0])
    max_y = np.max(indices_blancos[0])
    

    # Recortar el cuadrado blanco de la imagen original
    cuadrado_blanco = imagen_original.crop((min_x, min_y, max_x, max_y))
    # Obtener las dimensiones de la imagen recortada
    ancho, alto = cuadrado_blanco.size
    # Redimensionar el cuadrado blanco manteniendo la proporción
    max_ancho = ancho*2
    max_alto= alto*2 
    
     # Tamaño máximo deseado
    cuadrado_blanco=cuadrado_blanco.resize((max_ancho, max_alto))
    # Obtener las dimensiones de la imagen recortada
    ancho, alto = cuadrado_blanco.size
    
    
    return cuadrado_blanco
          
  except Exception as e:
      print('4 Error: {}'.format(e))
      return None
  

 

## Se configura cloudinary, estos valores se encuentran en el dashboard de la plataforma.

In [ ]:
import cloudinary

cloudinary.config(
      cloud_name = "dgsogjah1",
      api_key = "322113628466118",
      api_secret = "ifD7GZ9dYZ1j5ddABa_8gCOBVlI",
      )


## Funcion para descargar la imagen mediante una requests, retorna la imagen siendo un objeto de tipo Pillow. 

In [ ]:
def download_image_cloudinary(image_url):
  try:
      response = requests.get(image_url)
      status_code = response.status_code
      
      if status_code == 200:
          pass
          #print(f"La solicitud fue exitosa. Código de estado: {status_code}")
      else:
          print(f"La solicitud no fue exitosa. Código de estado: {status_code} {image_url}")
          raise Exception(f"La solicitud no fue exitosa. Código de estado: {status_code}")
      
      imagen_original = Image.open(BytesIO(response.content))
      #imagen_original.show()
      return imagen_original
      
  except Exception as e:
      print('3 Error: {}'.format(e))
      return None
  

## Funcion que se utiliza para subir la imagen en cloudinary, y retorna la respuesta.

In [ ]:
from cloudinary import uploader

def uploader_clodinary(file,public_id):
  try:
    # Cargar la imagen modificada a Cloudinary
    resultado = uploader.upload(
        file=file,
        public_id=public_id,
        )
    
    return resultado
  except Exception as e:
      print('2 Error: {}'.format(e))
      return False




## Funcion que toma  por argumento el nombre de una carpeta en existente en cloudinary, obtiene cada imagen y la modifica usando las anteriores funciones.

In [ ]:
from cloudinary import api

def get_folder_metadata(folder):
    try:
        resources = []
        next_cursor = None
        max_results = 500  # Número máximo de recursos por página
        
        while True:
            result = api.resources(
                type="upload",
                prefix=folder,
                max_results=max_results,
                next_cursor=next_cursor
            )
            resources.extend(result["resources"])
            
            if "next_cursor" not in result:
                break
            
            next_cursor = result["next_cursor"]
        
        total_resources = len(resources)
        print(f"Total de recursos encontrados en la carpeta: {total_resources}")
        

        for resource in resources:
            public_id = resource["public_id"]
            secure_url = resource["secure_url"]
            folder=resource["folder"]
            print(f'{folder} {public_id}')

            image=download_image_cloudinary(secure_url)

            if image:
              image=resize_image(image)
              if image:
                
                
                try:
                
                  with BytesIO() as buffer:
                    image.save(buffer, format="JPEG")
                    buffer.seek(0)
                    image_bytes = buffer.read()
                except Exception as e:
                  print(f"5 Error: {e}")
                  return False

                
                result=uploader_clodinary(image_bytes,public_id)

                if result:
                  #print(f'si {folder} {public_id}')
                  pass
                else:
                  print(f'no {folder} {public_id}')
            print('\n')

            # Realiza las operaciones deseadas con cada recurso
        
        return True
    
    except Exception as e:
        print(f"1 Error: {e}")
        return False



### Celda para ejecutar la funcion get_folder_metada, se la pasa como parametro el nombre de la carptea, solo descomentar cuando se requira ejecutar y despues volcer a comentar.

In [ ]:
#get_folder_metadata('rdn')

## Funcion para actualizar una sola imagen, se le pasa la url de cloudinary y su public id, y muestra al final la nueva ruta de la imagen.

In [ ]:
def only_one_image(secure_url,public_id):

  image=download_image_cloudinary(secure_url)
  image=resize_image(image)

  with BytesIO() as buffer:
    image.save(buffer, format="JPEG")
    buffer.seek(0)
    image_bytes = buffer.read()

  result=uploader_clodinary(image_bytes,public_id)
  print(result)


In [ ]:
secure_url='https://res.cloudinary.com/dgsogjah1/image/upload/v1685565427/noveplast/10022/10022-PACK-10-ARUBA-400x267.jpg'
public_id='noveplast/10023/10023-PACK-12-ARUBA-400x267'

#only_one_image(secure_url,public_id)


## Script que toma lee un archivo xlsx, toma la columna url_resource y modifica la imagen y agrega un columna con la nueva url, al final guarda el archivo modificado

In [ ]:

import pandas as pd

df = pd.read_excel('kendy.xlsx')
df['new_url'] = ''
for index, row in df.iterrows():
    # Acceder a los valores de cada columna en la fila actual
    col1_value = row['sku']
    col2_value = row['name'] 
    col3_value = row['url_resource'] 

    ruta_deseada=col3_value.split('/')
    nombre=ruta_deseada[-1]
    nombre=nombre.split('.')[0]
    public_id = '/'.join(ruta_deseada[7:-1]+[nombre])

    image=download_image_cloudinary(col3_value)
    if image:
      image=resize_image(image)
      if image:
        
        try:
          with BytesIO() as buffer:
            image.save(buffer, format="JPEG")
            buffer.seek(0)
            image_bytes = buffer.read()

            result=uploader_clodinary(image_bytes,public_id)
            df.at[index, 'new_url'] = result['secure_url']
            if result:
              pass
            else:
              print(f'no {col2_value} {col1_value}')

        except Exception as e:
          print(f"5 Error: {e} {col2_value} {col1_value}")
      print('\n')

# Guardar el DataFrame actualizado en un archivo XLSX
df.head()
df.to_excel('kendy_actualizado.xlsx', index=False)




















































































































































































































































5 Error: cannot write mode P as JPEG Broadway 65l 770229




/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


















































































































































































































5 Error: cannot write mode P as JPEG Tayrona No.1 baja 71590








5 Error: cannot write mode P as JPEG Broadway 65l 770229










































































































































5 Error: cannot write mode P as JPEG Jarra Mixer 3 Jarra Mixer 3










5 Error: cannot write mode P as JPEG Jarra Kool 3 Jarra Kool 3


5 Error: cannot write mode P as JPEG Jarra flip 3 Jarra flip 3


































































































































